<a href="https://colab.research.google.com/github/wlg100/numseqcont_circuit_expms/blob/main/notebook_templates/minimal_circuit_template.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" align="left"/></a>&nbsp;or in a local notebook.

# Setup
(No need to change anything)

In [1]:
# Janky code to do different setup when run in a Colab notebook vs VSCode
DEBUG_MODE = False
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
    %pip install git+https://github.com/neelnanda-io/TransformerLens.git
    # Install another version of node that makes PySvelte work way faster
    # !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
    # %pip install git+https://github.com/neelnanda-io/PySvelte.git
except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

Running as a Colab notebook
  Cloning https://github.com/neelnanda-io/TransformerLens.git to /tmp/pip-req-build-fi5bpo2m
  Running command git clone --filter=blob:none --quiet https://github.com/neelnanda-io/TransformerLens.git /tmp/pip-req-build-fi5bpo2m
  Resolved https://github.com/neelnanda-io/TransformerLens.git to commit fa287750606075574df2c538058e67d648e2f952
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 97.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 2.5 MB/s eta 0:00:00
    

In [2]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio

if IN_COLAB or not DEBUG_MODE:
    # Thanks to annoying rendering issues, Plotly graphics will either show up in colab OR Vscode depending on the renderer - this is bad for developing demos! Thus creating a debug mode.
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "png"

In [3]:
# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from fancy_einsum import einsum
import tqdm.notebook as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from jaxtyping import Float, Int
from typing import List, Union, Optional
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML

In [4]:
# import pysvelte

import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

We turn automatic differentiation off, to save GPU memory, as this notebook focuses on model inference not model training.

In [5]:
torch.set_grad_enabled(False)

Plotting helper functions:

In [6]:
def imshow(tensor, renderer=None, **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", **kwargs).show(renderer)

def line(tensor, renderer=None, **kwargs):
    px.line(y=utils.to_numpy(tensor), **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

## Load Model

Decide which model to use (eg. gpt2-small vs -medium)

In [7]:
model = HookedTransformer.from_pretrained(
    "gpt2-small",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=True,
)

Loaded pretrained model gpt2-small into HookedTransformer


## Import functions from repo

In [8]:
!git clone https://github.com/callummcdougall/ARENA_2.0.git

Cloning into 'ARENA_2.0'...
remote: Enumerating objects: 9165, done.
remote: Counting objects: 100% (1879/1879), done.
remote: Compressing objects: 100% (320/320), done.
remote: Total 9165 (delta 1652), reused 1656 (delta 1555), pack-reused 7286
Receiving objects: 100% (9165/9165), 156.33 MiB | 15.93 MiB/s, done.
Resolving deltas: 100% (5545/5545), done.


In [9]:
cd ARENA_2.0/chapter1_transformers/exercises/part3_indirect_object_identification

/content/ARENA_2.0/chapter1_transformers/exercises/part3_indirect_object_identification


In [10]:
import ioi_circuit_extraction as ioi_circuit_extraction

# test prompts

In [ ]:
example_prompt = "100 99 98 97"
example_answer = " 96"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '100', ' 99', ' 98', ' 97']
Tokenized answer: [' 96']


Performance on answer token:
Rank: 0        Logit: 20.37 Prob: 54.17% Token: | 96|

Top 0th token. Logit: 20.37 Prob: 54.17% Token: | 96|
Top 1th token. Logit: 18.67 Prob:  9.82% Token: | 97|
Top 2th token. Logit: 18.34 Prob:  7.09% Token: | 98|
Top 3th token. Logit: 18.20 Prob:  6.13% Token: | 95|
Top 4th token. Logit: 17.48 Prob:  2.98% Token: |
|
Top 5th token. Logit: 17.35 Prob:  2.62% Token: | 94|
Top 6th token. Logit: 17.34 Prob:  2.61% Token: | 92|
Top 7th token. Logit: 16.81 Prob:  1.53% Token: |.|
Top 8th token. Logit: 16.74 Prob:  1.44% Token: | 100|
Top 9th token. Logit: 16.33 Prob:  0.95% Token: | 99|


Ranks of the answer tokens: [(' 96', 0)]

In [ ]:
example_prompt = "58 57 56 55"
example_answer = " 54"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '58', ' 57', ' 56', ' 55']
Tokenized answer: [' 54']


Performance on answer token:
Rank: 0        Logit: 18.82 Prob: 26.37% Token: | 54|

Top 0th token. Logit: 18.82 Prob: 26.37% Token: | 54|
Top 1th token. Logit: 17.63 Prob:  8.06% Token: | 56|
Top 2th token. Logit: 17.37 Prob:  6.18% Token: | 55|
Top 3th token. Logit: 17.30 Prob:  5.77% Token: | 44|
Top 4th token. Logit: 17.16 Prob:  5.04% Token: | 52|
Top 5th token. Logit: 16.99 Prob:  4.23% Token: | 53|
Top 6th token. Logit: 16.88 Prob:  3.80% Token: | 46|
Top 7th token. Logit: 16.52 Prob:  2.64% Token: |
|
Top 8th token. Logit: 16.51 Prob:  2.63% Token: | 51|
Top 9th token. Logit: 16.45 Prob:  2.48% Token: | 50|


Ranks of the answer tokens: [(' 54', 0)]

In [ ]:
example_prompt = "104 103 102 101"
example_answer = " 100"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '104', ' 103', ' 102', ' 101']
Tokenized answer: [' 100']


Performance on answer token:
Rank: 0        Logit: 18.19 Prob: 23.47% Token: | 100|

Top 0th token. Logit: 18.19 Prob: 23.47% Token: | 100|
Top 1th token. Logit: 17.87 Prob: 17.05% Token: | 101|
Top 2th token. Logit: 17.14 Prob:  8.23% Token: | 102|
Top 3th token. Logit: 16.69 Prob:  5.24% Token: | 103|
Top 4th token. Logit: 16.57 Prob:  4.64% Token: |
|
Top 5th token. Logit: 16.54 Prob:  4.52% Token: | 99|
Top 6th token. Logit: 16.42 Prob:  4.01% Token: | 90|
Top 7th token. Logit: 16.30 Prob:  3.55% Token: | 98|
Top 8th token. Logit: 15.79 Prob:  2.14% Token: | 97|
Top 9th token. Logit: 15.75 Prob:  2.05% Token: | 0|


Ranks of the answer tokens: [(' 100', 0)]

In [ ]:
example_prompt = "105 104 103 102 101"
example_answer = " 100"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '105', ' 104', ' 103', ' 102', ' 101']
Tokenized answer: [' 100']


Performance on answer token:
Rank: 0        Logit: 19.31 Prob: 30.24% Token: | 100|

Top 0th token. Logit: 19.31 Prob: 30.24% Token: | 100|
Top 1th token. Logit: 18.95 Prob: 21.14% Token: | 101|
Top 2th token. Logit: 17.99 Prob:  8.10% Token: | 90|
Top 3th token. Logit: 17.71 Prob:  6.11% Token: | 99|
Top 4th token. Logit: 17.51 Prob:  5.05% Token: | 102|
Top 5th token. Logit: 17.35 Prob:  4.27% Token: |
|
Top 6th token. Logit: 16.84 Prob:  2.58% Token: | 98|
Top 7th token. Logit: 16.43 Prob:  1.70% Token: | 103|
Top 8th token. Logit: 16.36 Prob:  1.60% Token: | 97|
Top 9th token. Logit: 16.32 Prob:  1.53% Token: | 91|


Ranks of the answer tokens: [(' 100', 0)]

In [ ]:
example_prompt = "110 109 108 107 106 105 104 103"
example_answer = " 102"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '110', ' 109', ' 108', ' 107', ' 106', ' 105', ' 104', ' 103']
Tokenized answer: [' 102']


Performance on answer token:
Rank: 0        Logit: 23.03 Prob: 83.55% Token: | 102|

Top 0th token. Logit: 23.03 Prob: 83.55% Token: | 102|
Top 1th token. Logit: 20.43 Prob:  6.24% Token: | 103|
Top 2th token. Logit: 19.71 Prob:  3.04% Token: | 104|
Top 3th token. Logit: 18.68 Prob:  1.08% Token: | 101|
Top 4th token. Logit: 18.67 Prob:  1.08% Token: |
|
Top 5th token. Logit: 18.49 Prob:  0.89% Token: | 92|
Top 6th token. Logit: 18.16 Prob:  0.64% Token: | 100|
Top 7th token. Logit: 17.38 Prob:  0.29% Token: | 98|
Top 8th token. Logit: 16.94 Prob:  0.19% Token: |

|
Top 9th token. Logit: 16.84 Prob:  0.17% Token: | Clean|


Ranks of the answer tokens: [(' 102', 0)]

In [ ]:
example_prompt = "110 109 108 107 106 105 104 103 102 101"
example_answer = " 100"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '110', ' 109', ' 108', ' 107', ' 106', ' 105', ' 104', ' 103', ' 102', ' 101']
Tokenized answer: [' 100']


Performance on answer token:
Rank: 0        Logit: 20.99 Prob: 37.51% Token: | 100|

Top 0th token. Logit: 20.99 Prob: 37.51% Token: | 100|
Top 1th token. Logit: 20.53 Prob: 23.72% Token: | 101|
Top 2th token. Logit: 19.97 Prob: 13.60% Token: | 90|
Top 3th token. Logit: 19.03 Prob:  5.29% Token: |
|
Top 4th token. Logit: 18.63 Prob:  3.56% Token: | 99|
Top 5th token. Logit: 17.86 Prob:  1.65% Token: | 98|
Top 6th token. Logit: 17.61 Prob:  1.28% Token: |<|endoftext|>|
Top 7th token. Logit: 17.60 Prob:  1.26% Token: | 102|
Top 8th token. Logit: 17.14 Prob:  0.80% Token: | 0|
Top 9th token. Logit: 17.03 Prob:  0.72% Token: | 89|


Ranks of the answer tokens: [(' 100', 0)]

In [ ]:
example_prompt = "110 109 108 107 106 105 104 103 102 101 100"
example_answer = " 99"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '110', ' 109', ' 108', ' 107', ' 106', ' 105', ' 104', ' 103', ' 102', ' 101', ' 100']
Tokenized answer: [' 99']


Performance on answer token:
Rank: 0        Logit: 19.51 Prob: 55.29% Token: | 99|

Top 0th token. Logit: 19.51 Prob: 55.29% Token: | 99|
Top 1th token. Logit: 17.54 Prob:  7.74% Token: | 89|
Top 2th token. Logit: 17.47 Prob:  7.23% Token: |
|
Top 3th token. Logit: 17.06 Prob:  4.82% Token: | 100|
Top 4th token. Logit: 16.38 Prob:  2.42% Token: | 97|
Top 5th token. Logit: 15.98 Prob:  1.63% Token: |<|endoftext|>|
Top 6th token. Logit: 15.97 Prob:  1.62% Token: | No|
Top 7th token. Logit: 15.69 Prob:  1.22% Token: | 101|
Top 8th token. Logit: 15.59 Prob:  1.11% Token: | Top|
Top 9th token. Logit: 15.43 Prob:  0.93% Token: | 0|


Ranks of the answer tokens: [(' 99', 0)]

In [ ]:
example_prompt = "107 106 105 104 103 102 101 100"
example_answer = " 99"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '107', ' 106', ' 105', ' 104', ' 103', ' 102', ' 101', ' 100']
Tokenized answer: [' 99']


Performance on answer token:
Rank: 0        Logit: 18.19 Prob: 37.24% Token: | 99|

Top 0th token. Logit: 18.19 Prob: 37.24% Token: | 99|
Top 1th token. Logit: 17.29 Prob: 15.08% Token: | 100|
Top 2th token. Logit: 16.55 Prob:  7.25% Token: | 89|
Top 3th token. Logit: 16.51 Prob:  6.97% Token: |
|
Top 4th token. Logit: 16.10 Prob:  4.62% Token: | 97|
Top 5th token. Logit: 15.36 Prob:  2.20% Token: | 101|
Top 6th token. Logit: 15.00 Prob:  1.54% Token: | 98|
Top 7th token. Logit: 14.98 Prob:  1.50% Token: | No|
Top 8th token. Logit: 14.84 Prob:  1.31% Token: | 0|
Top 9th token. Logit: 14.32 Prob:  0.78% Token: | 91|


Ranks of the answer tokens: [(' 99', 0)]

In [ ]:
example_prompt = "107 106 105 104 103 102 101 100 99"
example_answer = " 98"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '107', ' 106', ' 105', ' 104', ' 103', ' 102', ' 101', ' 100', ' 99']
Tokenized answer: [' 98']


Performance on answer token:
Rank: 0        Logit: 21.75 Prob: 58.19% Token: | 98|

Top 0th token. Logit: 21.75 Prob: 58.19% Token: | 98|
Top 1th token. Logit: 20.03 Prob: 10.42% Token: | 100|
Top 2th token. Logit: 19.41 Prob:  5.60% Token: | 96|
Top 3th token. Logit: 19.40 Prob:  5.56% Token: | 88|
Top 4th token. Logit: 19.10 Prob:  4.09% Token: | 99|
Top 5th token. Logit: 18.92 Prob:  3.45% Token: | 97|
Top 6th token. Logit: 17.81 Prob:  1.13% Token: |.|
Top 7th token. Logit: 17.78 Prob:  1.10% Token: | 89|
Top 8th token. Logit: 17.78 Prob:  1.10% Token: | 92|
Top 9th token. Logit: 17.69 Prob:  1.00% Token: | 80|


Ranks of the answer tokens: [(' 98', 0)]

In [ ]:
example_prompt = "99 98 97 96 95"
example_answer = " 94"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '99', ' 98', ' 97', ' 96', ' 95']
Tokenized answer: [' 94']


Performance on answer token:
Rank: 0        Logit: 22.24 Prob: 54.84% Token: | 94|

Top 0th token. Logit: 22.24 Prob: 54.84% Token: | 94|
Top 1th token. Logit: 20.41 Prob:  8.81% Token: | 95|
Top 2th token. Logit: 20.33 Prob:  8.09% Token: | 92|
Top 3th token. Logit: 20.21 Prob:  7.17% Token: | 96|
Top 4th token. Logit: 20.17 Prob:  6.94% Token: | 93|
Top 5th token. Logit: 18.78 Prob:  1.73% Token: | 91|
Top 6th token. Logit: 18.49 Prob:  1.29% Token: |
|
Top 7th token. Logit: 18.41 Prob:  1.19% Token: | 90|
Top 8th token. Logit: 18.26 Prob:  1.02% Token: | 89|
Top 9th token. Logit: 18.15 Prob:  0.92% Token: | 84|


Ranks of the answer tokens: [(' 94', 0)]

In [ ]:
example_prompt = "99 98 97 96 95 94 93 92"
example_answer = " 91"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '99', ' 98', ' 97', ' 96', ' 95', ' 94', ' 93', ' 92']
Tokenized answer: [' 91']


Performance on answer token:
Rank: 0        Logit: 22.33 Prob: 80.08% Token: | 91|

Top 0th token. Logit: 22.33 Prob: 80.08% Token: | 91|
Top 1th token. Logit: 19.94 Prob:  7.30% Token: | 93|
Top 2th token. Logit: 19.92 Prob:  7.19% Token: | 92|
Top 3th token. Logit: 17.81 Prob:  0.87% Token: | 90|
Top 4th token. Logit: 17.76 Prob:  0.82% Token: | 89|
Top 5th token. Logit: 17.25 Prob:  0.50% Token: |
|
Top 6th token. Logit: 16.71 Prob:  0.29% Token: | 87|
Top 7th token. Logit: 16.01 Prob:  0.14% Token: | 71|
Top 8th token. Logit: 15.88 Prob:  0.13% Token: | Any|
Top 9th token. Logit: 15.66 Prob:  0.10% Token: | 1991|


Ranks of the answer tokens: [(' 91', 0)]

In [ ]:
example_prompt = "55 56 57 58"
example_answer = " 59"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '55', ' 56', ' 57', ' 58']
Tokenized answer: [' 59']


Performance on answer token:
Rank: 0        Logit: 18.23 Prob: 90.82% Token: | 59|

Top 0th token. Logit: 18.23 Prob: 90.82% Token: | 59|
Top 1th token. Logit: 14.05 Prob:  1.38% Token: | 69|
Top 2th token. Logit: 13.25 Prob:  0.62% Token: |
|
Top 3th token. Logit: 13.05 Prob:  0.51% Token: | 58|
Top 4th token. Logit: 12.74 Prob:  0.37% Token: | 61|
Top 5th token. Logit: 12.68 Prob:  0.35% Token: | 60|
Top 6th token. Logit: 12.36 Prob:  0.26% Token: | 57|
Top 7th token. Logit: 12.35 Prob:  0.25% Token: | 63|
Top 8th token. Logit: 11.97 Prob:  0.17% Token: | 49|
Top 9th token. Logit: 11.92 Prob:  0.16% Token: | 19|


Ranks of the answer tokens: [(' 59', 0)]

# Generate dataset with multiple prompts

In [11]:
class Dataset:
    def __init__(self, prompts, pos_dict, tokenizer, S1_is_first=False):
        self.prompts = prompts
        self.tokenizer = tokenizer
        self.N = len(prompts)
        self.max_len = max(
            [
                len(self.tokenizer(prompt["text"]).input_ids)
                for prompt in self.prompts
            ]
        )
        # all_ids = [prompt["TEMPLATE_IDX"] for prompt in self.ioi_prompts]
        all_ids = [0 for prompt in self.prompts] # only 1 template
        all_ids_ar = np.array(all_ids)
        self.groups = []
        for id in list(set(all_ids)):
            self.groups.append(np.where(all_ids_ar == id)[0])

        texts = [ prompt["text"] for prompt in self.prompts ]
        self.toks = torch.Tensor(self.tokenizer(texts, padding=True).input_ids).type(
            torch.int
        )
        self.io_tokenIDs = [
            self.tokenizer.encode(" " + prompt["corr"])[0] for prompt in self.prompts
        ]
        self.s_tokenIDs = [
            self.tokenizer.encode(" " + prompt["incorr"])[0] for prompt in self.prompts
        ]

        # word_idx: for every prompt, find the token index of each target token and "end"
        # word_idx is a tensor with an element for each prompt. The element is the targ token's ind at that prompt
        self.word_idx = {}
        for targ in [key for key in self.prompts[0].keys() if (key != 'text' and key != 'corr' and key != 'incorr')]:
            targ_lst = []
            for prompt in self.prompts:
                input_text = prompt["text"]
                tokens = model.tokenizer.tokenize(input_text)
                # if S1_is_first and targ == "S1":  # only use this if first token doesn't have space Ġ in front
                #     target_token = prompt[targ]
                # else:
                #     target_token = "Ġ" + prompt[targ]
                # target_index = tokens.index(target_token)
                target_index = pos_dict[targ]
                targ_lst.append(target_index)
            self.word_idx[targ] = torch.tensor(targ_lst)

        targ_lst = []
        for prompt in self.prompts:
            input_text = prompt["text"]
            tokens = self.tokenizer.tokenize(input_text)
            end_token_index = len(tokens) - 1
            targ_lst.append(end_token_index)
        self.word_idx["end"] = torch.tensor(targ_lst)

    def __len__(self):
        return self.N

NOTE: if the predicted token is not at high logit, then mean ablation can potentially lead to higher scores. Make sure the predicted token is near high 90s logit.

With 4 elems, gpt-2 small usually only gets to 50% for the correct token for decr, unlike incr, for 4 member seq. Often takes seqs of 10 elems to get to mid-90s (which is 4 seq of incr). So use 10 elems. See:

https://colab.research.google.com/drive/1ahWI9e0NMeAjdFNnj2vEj4d4aYIGsoNP#scrollTo=Nkbv00d0Wn2z&line=1&uniqifier=1

In [12]:
for i in range(21, 17, -1):
    print(i)

21
20
19
18


In [13]:
pos_dict = {
    'S1': 0,
    'S2': 1,
    'S3': 2,
    'S4': 3,
    'S5': 4,
    'S6': 5,
    'S7': 6,
    'S8': 7,
}

In [14]:
def generate_prompts_list(x ,y):
    prompts_list = []
    for i in range(x, y, -1):
        prompt_dict = {
            'S1': str(i),
            'S2': str(i-1),
            'S3': str(i-2),
            'S4': str(i-3),
            'S5': str(i-4),
            'S6': str(i-5),
            'S7': str(i-6),
            'S8': str(i-7),
            'corr': str(i-8),
            'incorr': str(i-7),
            'text': f"{i} {i-1} {i-2} {i-3} {i-4} {i-5} {i-6} {i-7}"
        }
        prompts_list.append(prompt_dict)
    return prompts_list

prompts_list = generate_prompts_list(100, 8)
dataset = Dataset(prompts_list, pos_dict, model.tokenizer, S1_is_first=True)
len(prompts_list)

92

In [15]:
prompts_list[-1]

{'S1': '9',
 'S2': '8',
 'S3': '7',
 'S4': '6',
 'S5': '5',
 'S6': '4',
 'S7': '3',
 'S8': '2',
 'corr': '1',
 'incorr': '2',
 'text': '9 8 7 6 5 4 3 2'}

In [16]:
prompts_list_2 = [{'S1': '53',
  'S2': '8',
  'S3': '58',
  'S4': '57',
  'S5': '2',
  'S6': '4',
  'S7': '72',
  'S8': '57',
  'corr': '92',
  'incorr': '93',
  'text': '53 8 58 57 2 4 72 57'},
 {'S1': '93',
  'S2': '43',
  'S3': '43',
  'S4': '31',
  'S5': '3',
  'S6': '16',
  'S7': '59',
  'S8': '37',
  'corr': '91',
  'incorr': '92',
  'text': '93 43 43 31 3 16 59 37'},
 {'S1': '97',
  'S2': '55',
  'S3': '37',
  'S4': '63',
  'S5': '89',
  'S6': '47',
  'S7': '33',
  'S8': '82',
  'corr': '90',
  'incorr': '91',
  'text': '97 55 37 63 89 47 33 82'},
 {'S1': '14',
  'S2': '4',
  'S3': '59',
  'S4': '81',
  'S5': '70',
  'S6': '80',
  'S7': '10',
  'S8': '57',
  'corr': '89',
  'incorr': '90',
  'text': '14 4 59 81 70 80 10 57'},
 {'S1': '30',
  'S2': '80',
  'S3': '92',
  'S4': '16',
  'S5': '99',
  'S6': '15',
  'S7': '57',
  'S8': '17',
  'corr': '88',
  'incorr': '89',
  'text': '30 80 92 16 99 15 57 17'},
 {'S1': '80',
  'S2': '91',
  'S3': '90',
  'S4': '96',
  'S5': '50',
  'S6': '79',
  'S7': '70',
  'S8': '16',
  'corr': '87',
  'incorr': '88',
  'text': '80 91 90 96 50 79 70 16'},
 {'S1': '51',
  'S2': '75',
  'S3': '98',
  'S4': '96',
  'S5': '77',
  'S6': '97',
  'S7': '13',
  'S8': '23',
  'corr': '86',
  'incorr': '87',
  'text': '51 75 98 96 77 97 13 23'},
 {'S1': '31',
  'S2': '38',
  'S3': '96',
  'S4': '42',
  'S5': '8',
  'S6': '10',
  'S7': '69',
  'S8': '59',
  'corr': '85',
  'incorr': '86',
  'text': '31 38 96 42 8 10 69 59'},
 {'S1': '14',
  'S2': '15',
  'S3': '88',
  'S4': '76',
  'S5': '58',
  'S6': '56',
  'S7': '84',
  'S8': '53',
  'corr': '84',
  'incorr': '85',
  'text': '14 15 88 76 58 56 84 53'},
 {'S1': '4',
  'S2': '24',
  'S3': '96',
  'S4': '65',
  'S5': '38',
  'S6': '18',
  'S7': '91',
  'S8': '31',
  'corr': '83',
  'incorr': '84',
  'text': '4 24 96 65 38 18 91 31'},
 {'S1': '73',
  'S2': '77',
  'S3': '76',
  'S4': '53',
  'S5': '32',
  'S6': '44',
  'S7': '43',
  'S8': '11',
  'corr': '82',
  'incorr': '83',
  'text': '73 77 76 53 32 44 43 11'},
 {'S1': '83',
  'S2': '100',
  'S3': '99',
  'S4': '54',
  'S5': '94',
  'S6': '76',
  'S7': '11',
  'S8': '47',
  'corr': '81',
  'incorr': '82',
  'text': '83 100 99 54 94 76 11 47'},
 {'S1': '46',
  'S2': '23',
  'S3': '35',
  'S4': '51',
  'S5': '90',
  'S6': '25',
  'S7': '29',
  'S8': '12',
  'corr': '80',
  'incorr': '81',
  'text': '46 23 35 51 90 25 29 12'},
 {'S1': '39',
  'S2': '28',
  'S3': '28',
  'S4': '47',
  'S5': '14',
  'S6': '29',
  'S7': '44',
  'S8': '12',
  'corr': '79',
  'incorr': '80',
  'text': '39 28 28 47 14 29 44 12'},
 {'S1': '22',
  'S2': '36',
  'S3': '31',
  'S4': '56',
  'S5': '56',
  'S6': '7',
  'S7': '53',
  'S8': '62',
  'corr': '78',
  'incorr': '79',
  'text': '22 36 31 56 56 7 53 62'},
 {'S1': '84',
  'S2': '10',
  'S3': '56',
  'S4': '64',
  'S5': '9',
  'S6': '79',
  'S7': '63',
  'S8': '70',
  'corr': '77',
  'incorr': '78',
  'text': '84 10 56 64 9 79 63 70'},
 {'S1': '88',
  'S2': '72',
  'S3': '11',
  'S4': '15',
  'S5': '64',
  'S6': '60',
  'S7': '74',
  'S8': '90',
  'corr': '76',
  'incorr': '77',
  'text': '88 72 11 15 64 60 74 90'},
 {'S1': '32',
  'S2': '80',
  'S3': '51',
  'S4': '40',
  'S5': '41',
  'S6': '9',
  'S7': '25',
  'S8': '34',
  'corr': '75',
  'incorr': '76',
  'text': '32 80 51 40 41 9 25 34'},
 {'S1': '97',
  'S2': '26',
  'S3': '15',
  'S4': '81',
  'S5': '89',
  'S6': '92',
  'S7': '26',
  'S8': '59',
  'corr': '74',
  'incorr': '75',
  'text': '97 26 15 81 89 92 26 59'},
 {'S1': '9',
  'S2': '39',
  'S3': '37',
  'S4': '52',
  'S5': '53',
  'S6': '6',
  'S7': '18',
  'S8': '75',
  'corr': '73',
  'incorr': '74',
  'text': '9 39 37 52 53 6 18 75'},
 {'S1': '68',
  'S2': '60',
  'S3': '98',
  'S4': '43',
  'S5': '80',
  'S6': '83',
  'S7': '33',
  'S8': '35',
  'corr': '72',
  'incorr': '73',
  'text': '68 60 98 43 80 83 33 35'},
 {'S1': '98',
  'S2': '37',
  'S3': '27',
  'S4': '64',
  'S5': '64',
  'S6': '89',
  'S7': '26',
  'S8': '46',
  'corr': '71',
  'incorr': '72',
  'text': '98 37 27 64 64 89 26 46'},
 {'S1': '25',
  'S2': '17',
  'S3': '70',
  'S4': '51',
  'S5': '86',
  'S6': '83',
  'S7': '75',
  'S8': '16',
  'corr': '70',
  'incorr': '71',
  'text': '25 17 70 51 86 83 75 16'},
 {'S1': '98',
  'S2': '31',
  'S3': '62',
  'S4': '40',
  'S5': '76',
  'S6': '70',
  'S7': '80',
  'S8': '71',
  'corr': '69',
  'incorr': '70',
  'text': '98 31 62 40 76 70 80 71'},
 {'S1': '5',
  'S2': '82',
  'S3': '10',
  'S4': '69',
  'S5': '8',
  'S6': '69',
  'S7': '23',
  'S8': '2',
  'corr': '68',
  'incorr': '69',
  'text': '5 82 10 69 8 69 23 2'},
 {'S1': '65',
  'S2': '67',
  'S3': '70',
  'S4': '96',
  'S5': '51',
  'S6': '94',
  'S7': '9',
  'S8': '83',
  'corr': '67',
  'incorr': '68',
  'text': '65 67 70 96 51 94 9 83'},
 {'S1': '63',
  'S2': '24',
  'S3': '9',
  'S4': '44',
  'S5': '44',
  'S6': '93',
  'S7': '71',
  'S8': '100',
  'corr': '66',
  'incorr': '67',
  'text': '63 24 9 44 44 93 71 100'},
 {'S1': '100',
  'S2': '64',
  'S3': '48',
  'S4': '22',
  'S5': '46',
  'S6': '20',
  'S7': '72',
  'S8': '11',
  'corr': '65',
  'incorr': '66',
  'text': '100 64 48 22 46 20 72 11'},
 {'S1': '81',
  'S2': '3',
  'S3': '49',
  'S4': '55',
  'S5': '96',
  'S6': '50',
  'S7': '14',
  'S8': '5',
  'corr': '64',
  'incorr': '65',
  'text': '81 3 49 55 96 50 14 5'},
 {'S1': '12',
  'S2': '23',
  'S3': '29',
  'S4': '18',
  'S5': '29',
  'S6': '3',
  'S7': '14',
  'S8': '20',
  'corr': '63',
  'incorr': '64',
  'text': '12 23 29 18 29 3 14 20'},
 {'S1': '87',
  'S2': '54',
  'S3': '51',
  'S4': '81',
  'S5': '64',
  'S6': '37',
  'S7': '33',
  'S8': '53',
  'corr': '62',
  'incorr': '63',
  'text': '87 54 51 81 64 37 33 53'},
 {'S1': '12',
  'S2': '7',
  'S3': '70',
  'S4': '54',
  'S5': '15',
  'S6': '20',
  'S7': '30',
  'S8': '63',
  'corr': '61',
  'incorr': '62',
  'text': '12 7 70 54 15 20 30 63'},
 {'S1': '31',
  'S2': '28',
  'S3': '29',
  'S4': '98',
  'S5': '80',
  'S6': '21',
  'S7': '16',
  'S8': '94',
  'corr': '60',
  'incorr': '61',
  'text': '31 28 29 98 80 21 16 94'},
 {'S1': '67',
  'S2': '25',
  'S3': '94',
  'S4': '52',
  'S5': '38',
  'S6': '45',
  'S7': '99',
  'S8': '72',
  'corr': '59',
  'incorr': '60',
  'text': '67 25 94 52 38 45 99 72'},
 {'S1': '89',
  'S2': '41',
  'S3': '41',
  'S4': '62',
  'S5': '11',
  'S6': '50',
  'S7': '50',
  'S8': '78',
  'corr': '58',
  'incorr': '59',
  'text': '89 41 41 62 11 50 50 78'},
 {'S1': '94',
  'S2': '64',
  'S3': '34',
  'S4': '3',
  'S5': '78',
  'S6': '78',
  'S7': '22',
  'S8': '34',
  'corr': '57',
  'incorr': '58',
  'text': '94 64 34 3 78 78 22 34'},
 {'S1': '100',
  'S2': '32',
  'S3': '24',
  'S4': '54',
  'S5': '3',
  'S6': '18',
  'S7': '80',
  'S8': '72',
  'corr': '56',
  'incorr': '57',
  'text': '100 32 24 54 3 18 80 72'},
 {'S1': '77',
  'S2': '94',
  'S3': '54',
  'S4': '57',
  'S5': '63',
  'S6': '30',
  'S7': '34',
  'S8': '56',
  'corr': '55',
  'incorr': '56',
  'text': '77 94 54 57 63 30 34 56'},
 {'S1': '62',
  'S2': '33',
  'S3': '86',
  'S4': '83',
  'S5': '84',
  'S6': '28',
  'S7': '57',
  'S8': '39',
  'corr': '54',
  'incorr': '55',
  'text': '62 33 86 83 84 28 57 39'},
 {'S1': '40',
  'S2': '42',
  'S3': '11',
  'S4': '59',
  'S5': '48',
  'S6': '24',
  'S7': '6',
  'S8': '82',
  'corr': '53',
  'incorr': '54',
  'text': '40 42 11 59 48 24 6 82'},
 {'S1': '53',
  'S2': '88',
  'S3': '61',
  'S4': '88',
  'S5': '88',
  'S6': '90',
  'S7': '70',
  'S8': '73',
  'corr': '52',
  'incorr': '53',
  'text': '53 88 61 88 88 90 70 73'},
 {'S1': '22',
  'S2': '42',
  'S3': '11',
  'S4': '21',
  'S5': '36',
  'S6': '71',
  'S7': '33',
  'S8': '6',
  'corr': '51',
  'incorr': '52',
  'text': '22 42 11 21 36 71 33 6'},
 {'S1': '47',
  'S2': '43',
  'S3': '76',
  'S4': '57',
  'S5': '25',
  'S6': '37',
  'S7': '12',
  'S8': '59',
  'corr': '50',
  'incorr': '51',
  'text': '47 43 76 57 25 37 12 59'},
 {'S1': '13',
  'S2': '69',
  'S3': '42',
  'S4': '28',
  'S5': '24',
  'S6': '17',
  'S7': '100',
  'S8': '52',
  'corr': '49',
  'incorr': '50',
  'text': '13 69 42 28 24 17 100 52'},
 {'S1': '72',
  'S2': '21',
  'S3': '83',
  'S4': '17',
  'S5': '81',
  'S6': '50',
  'S7': '55',
  'S8': '77',
  'corr': '48',
  'incorr': '49',
  'text': '72 21 83 17 81 50 55 77'},
 {'S1': '91',
  'S2': '91',
  'S3': '42',
  'S4': '24',
  'S5': '31',
  'S6': '74',
  'S7': '2',
  'S8': '5',
  'corr': '47',
  'incorr': '48',
  'text': '91 91 42 24 31 74 2 5'},
 {'S1': '75',
  'S2': '49',
  'S3': '6',
  'S4': '3',
  'S5': '45',
  'S6': '59',
  'S7': '16',
  'S8': '21',
  'corr': '46',
  'incorr': '47',
  'text': '75 49 6 3 45 59 16 21'},
 {'S1': '31',
  'S2': '75',
  'S3': '68',
  'S4': '24',
  'S5': '80',
  'S6': '30',
  'S7': '91',
  'S8': '97',
  'corr': '45',
  'incorr': '46',
  'text': '31 75 68 24 80 30 91 97'},
 {'S1': '78',
  'S2': '89',
  'S3': '96',
  'S4': '32',
  'S5': '33',
  'S6': '80',
  'S7': '67',
  'S8': '72',
  'corr': '44',
  'incorr': '45',
  'text': '78 89 96 32 33 80 67 72'},
 {'S1': '78',
  'S2': '31',
  'S3': '94',
  'S4': '53',
  'S5': '59',
  'S6': '31',
  'S7': '17',
  'S8': '37',
  'corr': '43',
  'incorr': '44',
  'text': '78 31 94 53 59 31 17 37'},
 {'S1': '9',
  'S2': '49',
  'S3': '67',
  'S4': '59',
  'S5': '45',
  'S6': '69',
  'S7': '2',
  'S8': '86',
  'corr': '42',
  'incorr': '43',
  'text': '9 49 67 59 45 69 2 86'},
 {'S1': '73',
  'S2': '28',
  'S3': '15',
  'S4': '36',
  'S5': '34',
  'S6': '83',
  'S7': '23',
  'S8': '52',
  'corr': '41',
  'incorr': '42',
  'text': '73 28 15 36 34 83 23 52'},
 {'S1': '33',
  'S2': '28',
  'S3': '37',
  'S4': '19',
  'S5': '95',
  'S6': '50',
  'S7': '16',
  'S8': '55',
  'corr': '40',
  'incorr': '41',
  'text': '33 28 37 19 95 50 16 55'},
 {'S1': '21',
  'S2': '19',
  'S3': '46',
  'S4': '8',
  'S5': '91',
  'S6': '22',
  'S7': '58',
  'S8': '24',
  'corr': '39',
  'incorr': '40',
  'text': '21 19 46 8 91 22 58 24'},
 {'S1': '67',
  'S2': '41',
  'S3': '34',
  'S4': '74',
  'S5': '68',
  'S6': '4',
  'S7': '14',
  'S8': '77',
  'corr': '38',
  'incorr': '39',
  'text': '67 41 34 74 68 4 14 77'},
 {'S1': '63',
  'S2': '23',
  'S3': '16',
  'S4': '18',
  'S5': '70',
  'S6': '84',
  'S7': '76',
  'S8': '6',
  'corr': '37',
  'incorr': '38',
  'text': '63 23 16 18 70 84 76 6'},
 {'S1': '21',
  'S2': '13',
  'S3': '67',
  'S4': '36',
  'S5': '90',
  'S6': '43',
  'S7': '75',
  'S8': '69',
  'corr': '36',
  'incorr': '37',
  'text': '21 13 67 36 90 43 75 69'},
 {'S1': '23',
  'S2': '40',
  'S3': '54',
  'S4': '55',
  'S5': '39',
  'S6': '53',
  'S7': '85',
  'S8': '93',
  'corr': '35',
  'incorr': '36',
  'text': '23 40 54 55 39 53 85 93'},
 {'S1': '69',
  'S2': '84',
  'S3': '24',
  'S4': '20',
  'S5': '99',
  'S6': '22',
  'S7': '97',
  'S8': '12',
  'corr': '34',
  'incorr': '35',
  'text': '69 84 24 20 99 22 97 12'},
 {'S1': '42',
  'S2': '61',
  'S3': '88',
  'S4': '41',
  'S5': '90',
  'S6': '11',
  'S7': '54',
  'S8': '19',
  'corr': '33',
  'incorr': '34',
  'text': '42 61 88 41 90 11 54 19'},
 {'S1': '73',
  'S2': '29',
  'S3': '86',
  'S4': '8',
  'S5': '5',
  'S6': '51',
  'S7': '25',
  'S8': '91',
  'corr': '32',
  'incorr': '33',
  'text': '73 29 86 8 5 51 25 91'},
 {'S1': '16',
  'S2': '26',
  'S3': '34',
  'S4': '58',
  'S5': '68',
  'S6': '88',
  'S7': '61',
  'S8': '55',
  'corr': '31',
  'incorr': '32',
  'text': '16 26 34 58 68 88 61 55'},
 {'S1': '33',
  'S2': '9',
  'S3': '79',
  'S4': '11',
  'S5': '7',
  'S6': '53',
  'S7': '39',
  'S8': '23',
  'corr': '30',
  'incorr': '31',
  'text': '33 9 79 11 7 53 39 23'},
 {'S1': '38',
  'S2': '76',
  'S3': '79',
  'S4': '21',
  'S5': '94',
  'S6': '100',
  'S7': '21',
  'S8': '59',
  'corr': '29',
  'incorr': '30',
  'text': '38 76 79 21 94 100 21 59'},
 {'S1': '6',
  'S2': '66',
  'S3': '41',
  'S4': '38',
  'S5': '77',
  'S6': '52',
  'S7': '11',
  'S8': '11',
  'corr': '28',
  'incorr': '29',
  'text': '6 66 41 38 77 52 11 11'},
 {'S1': '26',
  'S2': '25',
  'S3': '63',
  'S4': '70',
  'S5': '13',
  'S6': '98',
  'S7': '39',
  'S8': '49',
  'corr': '27',
  'incorr': '28',
  'text': '26 25 63 70 13 98 39 49'},
 {'S1': '90',
  'S2': '43',
  'S3': '53',
  'S4': '10',
  'S5': '40',
  'S6': '85',
  'S7': '90',
  'S8': '4',
  'corr': '26',
  'incorr': '27',
  'text': '90 43 53 10 40 85 90 4'},
 {'S1': '16',
  'S2': '74',
  'S3': '20',
  'S4': '47',
  'S5': '11',
  'S6': '86',
  'S7': '78',
  'S8': '64',
  'corr': '25',
  'incorr': '26',
  'text': '16 74 20 47 11 86 78 64'},
 {'S1': '25',
  'S2': '20',
  'S3': '29',
  'S4': '35',
  'S5': '76',
  'S6': '82',
  'S7': '36',
  'S8': '77',
  'corr': '24',
  'incorr': '25',
  'text': '25 20 29 35 76 82 36 77'},
 {'S1': '4',
  'S2': '78',
  'S3': '97',
  'S4': '23',
  'S5': '3',
  'S6': '89',
  'S7': '65',
  'S8': '53',
  'corr': '23',
  'incorr': '24',
  'text': '4 78 97 23 3 89 65 53'},
 {'S1': '77',
  'S2': '98',
  'S3': '49',
  'S4': '59',
  'S5': '30',
  'S6': '44',
  'S7': '60',
  'S8': '36',
  'corr': '22',
  'incorr': '23',
  'text': '77 98 49 59 30 44 60 36'},
 {'S1': '44',
  'S2': '74',
  'S3': '6',
  'S4': '91',
  'S5': '6',
  'S6': '54',
  'S7': '94',
  'S8': '66',
  'corr': '21',
  'incorr': '22',
  'text': '44 74 6 91 6 54 94 66'},
 {'S1': '66',
  'S2': '42',
  'S3': '72',
  'S4': '38',
  'S5': '65',
  'S6': '47',
  'S7': '63',
  'S8': '16',
  'corr': '20',
  'incorr': '21',
  'text': '66 42 72 38 65 47 63 16'},
 {'S1': '66',
  'S2': '97',
  'S3': '46',
  'S4': '77',
  'S5': '40',
  'S6': '6',
  'S7': '81',
  'S8': '46',
  'corr': '19',
  'incorr': '20',
  'text': '66 97 46 77 40 6 81 46'},
 {'S1': '85',
  'S2': '85',
  'S3': '44',
  'S4': '94',
  'S5': '66',
  'S6': '19',
  'S7': '99',
  'S8': '62',
  'corr': '18',
  'incorr': '19',
  'text': '85 85 44 94 66 19 99 62'},
 {'S1': '61',
  'S2': '4',
  'S3': '91',
  'S4': '19',
  'S5': '29',
  'S6': '22',
  'S7': '46',
  'S8': '40',
  'corr': '17',
  'incorr': '18',
  'text': '61 4 91 19 29 22 46 40'},
 {'S1': '6',
  'S2': '84',
  'S3': '89',
  'S4': '50',
  'S5': '45',
  'S6': '84',
  'S7': '8',
  'S8': '50',
  'corr': '16',
  'incorr': '17',
  'text': '6 84 89 50 45 84 8 50'},
 {'S1': '18',
  'S2': '80',
  'S3': '74',
  'S4': '43',
  'S5': '17',
  'S6': '16',
  'S7': '84',
  'S8': '81',
  'corr': '15',
  'incorr': '16',
  'text': '18 80 74 43 17 16 84 81'},
 {'S1': '25',
  'S2': '11',
  'S3': '82',
  'S4': '64',
  'S5': '42',
  'S6': '78',
  'S7': '21',
  'S8': '62',
  'corr': '14',
  'incorr': '15',
  'text': '25 11 82 64 42 78 21 62'},
 {'S1': '75',
  'S2': '22',
  'S3': '58',
  'S4': '22',
  'S5': '80',
  'S6': '25',
  'S7': '68',
  'S8': '32',
  'corr': '13',
  'incorr': '14',
  'text': '75 22 58 22 80 25 68 32'},
 {'S1': '68',
  'S2': '88',
  'S3': '50',
  'S4': '20',
  'S5': '61',
  'S6': '78',
  'S7': '100',
  'S8': '34',
  'corr': '12',
  'incorr': '13',
  'text': '68 88 50 20 61 78 100 34'},
 {'S1': '85',
  'S2': '35',
  'S3': '10',
  'S4': '67',
  'S5': '43',
  'S6': '90',
  'S7': '86',
  'S8': '84',
  'corr': '11',
  'incorr': '12',
  'text': '85 35 10 67 43 90 86 84'},
 {'S1': '19',
  'S2': '11',
  'S3': '87',
  'S4': '61',
  'S5': '15',
  'S6': '36',
  'S7': '7',
  'S8': '39',
  'corr': '10',
  'incorr': '11',
  'text': '19 11 87 61 15 36 7 39'},
 {'S1': '3',
  'S2': '4',
  'S3': '19',
  'S4': '87',
  'S5': '66',
  'S6': '87',
  'S7': '24',
  'S8': '5',
  'corr': '9',
  'incorr': '10',
  'text': '3 4 19 87 66 87 24 5'},
 {'S1': '74',
  'S2': '37',
  'S3': '61',
  'S4': '64',
  'S5': '94',
  'S6': '72',
  'S7': '67',
  'S8': '30',
  'corr': '8',
  'incorr': '9',
  'text': '74 37 61 64 94 72 67 30'},
 {'S1': '71',
  'S2': '4',
  'S3': '6',
  'S4': '16',
  'S5': '40',
  'S6': '2',
  'S7': '69',
  'S8': '97',
  'corr': '7',
  'incorr': '8',
  'text': '71 4 6 16 40 2 69 97'},
 {'S1': '13',
  'S2': '77',
  'S3': '96',
  'S4': '3',
  'S5': '47',
  'S6': '49',
  'S7': '38',
  'S8': '82',
  'corr': '6',
  'incorr': '7',
  'text': '13 77 96 3 47 49 38 82'},
 {'S1': '9',
  'S2': '97',
  'S3': '71',
  'S4': '89',
  'S5': '35',
  'S6': '69',
  'S7': '81',
  'S8': '15',
  'corr': '5',
  'incorr': '6',
  'text': '9 97 71 89 35 69 81 15'},
 {'S1': '16',
  'S2': '53',
  'S3': '30',
  'S4': '50',
  'S5': '83',
  'S6': '10',
  'S7': '12',
  'S8': '22',
  'corr': '4',
  'incorr': '5',
  'text': '16 53 30 50 83 10 12 22'},
 {'S1': '14',
  'S2': '31',
  'S3': '28',
  'S4': '35',
  'S5': '54',
  'S6': '13',
  'S7': '33',
  'S8': '66',
  'corr': '3',
  'incorr': '4',
  'text': '14 31 28 35 54 13 33 66'},
 {'S1': '88',
  'S2': '16',
  'S3': '70',
  'S4': '97',
  'S5': '82',
  'S6': '42',
  'S7': '53',
  'S8': '3',
  'corr': '2',
  'incorr': '3',
  'text': '88 16 70 97 82 42 53 3'},
 {'S1': '18',
  'S2': '40',
  'S3': '45',
  'S4': '13',
  'S5': '95',
  'S6': '7',
  'S7': '63',
  'S8': '72',
  'corr': '1',
  'incorr': '2',
  'text': '18 40 45 13 95 7 63 72'}]

In [17]:
# import random

# def generate_prompts_list_corr(x ,y):
#     prompts_list = []
#     for i in range(x, y, -1):
#         r1 = random.randint(1, 100)
#         r2 = random.randint(1, 100)
#         r3 = random.randint(1, 100)
#         r4 = random.randint(1, 100)
#         r4 = random.randint(1, 100)
#         r5 = random.randint(1, 100)
#         r6 = random.randint(1, 100)
#         while True:
#             r7 = random.randint(1, 100)
#             r8 = random.randint(1, 100)
#             if r7 -1 != r8:
#                 break
#         prompt_dict = {
#             'S1': str(r1),
#             'S2': str(r2),
#             'S3': str(r3),
#             'S4': str(r4),
#             'S5': str(r5),
#             'S6': str(r6),
#             'S7': str(r7),
#             'S8': str(r8),
#             'corr': str(i-8),
#             'incorr': str(i-7),
#             'text': f"{r1} {r2} {r3} {r4} {r5} {r6} {r7} {r8}"
#         }
#         prompts_list.append(prompt_dict)
#     return prompts_list

# prompts_list_2 = generate_prompts_list_corr(100, 8)
dataset_2 = Dataset(prompts_list_2, pos_dict, model.tokenizer, S1_is_first=True)
# prompts_list_2

# Ablation Expm Functions

In [18]:
from torch import Tensor

def logits_to_ave_logit_diff_2(logits: Float[Tensor, "batch seq d_vocab"], dataset: Dataset, per_prompt=False):
    '''
    Returns logit difference between the correct and incorrect answer.

    If per_prompt=True, return the array of differences rather than the average.
    '''

    # Only the final logits are relevant for the answer
    # Get the logits corresponding to the indirect object / subject tokens respectively
    io_logits: Float[Tensor, "batch"] = logits[range(logits.size(0)), dataset.word_idx["end"], dataset.io_tokenIDs]
    s_logits: Float[Tensor, "batch"] = logits[range(logits.size(0)), dataset.word_idx["end"], dataset.s_tokenIDs]
    # Find logit difference
    answer_logit_diff = io_logits - s_logits
    return answer_logit_diff if per_prompt else answer_logit_diff.mean()

In [19]:
model.reset_hooks(including_permanent=True)  #must do this after running with mean ablation hook
ioi_logits_original, ioi_cache = model.run_with_cache(dataset.toks)
orig_score = logits_to_ave_logit_diff_2(ioi_logits_original, dataset)

In [20]:
def mean_ablate_by_lst(lst, model, orig_score, print_output=True):
    CIRCUIT = {
        "number mover": lst,
        # "number mover 10": lst,
        # "number mover 9": lst,
        "number mover 8": lst,
        "number mover 7": lst,
        "number mover 6": lst,
        "number mover 5": lst,
        "number mover 4": lst,
        "number mover 3": lst,
        "number mover 2": lst,
        "number mover 1": lst,
    }

    SEQ_POS_TO_KEEP = {
        "number mover": "end",
        # "number mover 10": "S10",
        # "number mover 9": "S9",
        "number mover 8": "S8",
        "number mover 7": "S7",
        "number mover 6": "S6",
        "number mover 5": "S5",
        "number mover 4": "S4",
        "number mover 3": "S3",
        "number mover 2": "S2",
        "number mover 1": "S1",
    }

    model.reset_hooks(including_permanent=True)  #must do this after running with mean ablation hook

    # ioi_logits_original, ioi_cache = model.run_with_cache(dataset.toks)

    model = ioi_circuit_extraction.add_mean_ablation_hook(model, means_dataset=dataset_2, circuit=CIRCUIT, seq_pos_to_keep=SEQ_POS_TO_KEEP)
    ioi_logits_minimal = model(dataset.toks)

    # orig_score = logits_to_ave_logit_diff_2(ioi_logits_original, dataset)
    new_score = logits_to_ave_logit_diff_2(ioi_logits_minimal, dataset)
    if print_output:
        # print(f"Average logit difference (IOI dataset, using entire model): {orig_score:.4f}")
        # print(f"Average logit difference (IOI dataset, only using circuit): {new_score:.4f}")
        print(f"Average logit difference (circuit / full) %: {100 * new_score / orig_score:.4f}")
    # return new_score
    return 100 * new_score / orig_score

In [21]:
curr_circuit = [(layer, head) for layer in range(12) for head in range(12)]
mean_ablate_by_lst(curr_circuit, model, orig_score, print_output=False).item()

100.0

In [22]:
def find_circuit_forw(curr_circuit=None, orig_score=orig_score, threshold=10):
    # threshold is T, a %. if performance is less than T%, allow its removal
    if curr_circuit == []:
        # Start with full circuit
        curr_circuit = [(layer, head) for layer in range(12) for head in range(12)]

    for layer in range(0, 12):
        for head in range(12):
            if (layer, head) not in curr_circuit:
                continue

            # Copying the curr_circuit so we can iterate over one and modify the other
            copy_circuit = curr_circuit.copy()

            # Temporarily removing the current tuple from the copied circuit
            copy_circuit.remove((layer, head))

            new_score = mean_ablate_by_lst(copy_circuit, model, orig_score, print_output=False).item()

            # print((layer,head), new_score)
            # If the result is less than the threshold, remove the tuple from the original list
            if (100 - new_score) < threshold:
                curr_circuit.remove((layer, head))

                print("\nRemoved:", (layer, head))
                print(new_score)

    return curr_circuit, new_score

In [23]:
def find_circuit_backw(curr_circuit=None, orig_score=orig_score, threshold=10):
    # threshold is T, a %. if performance is less than T%, allow its removal
    if curr_circuit == []:
        # Start with full circuit
        curr_circuit = [(layer, head) for layer in range(12) for head in range(12)]

    for layer in range(11, -1, -1):  # go thru all heads in a layer first
        for head in range(12):
            if (layer, head) not in curr_circuit:
                continue

            # Copying the curr_circuit so we can iterate over one and modify the other
            copy_circuit = curr_circuit.copy()

            # Temporarily removing the current tuple from the copied circuit
            copy_circuit.remove((layer, head))

            new_score = mean_ablate_by_lst(copy_circuit, model, orig_score, print_output=False).item()

            # If the result is less than the threshold, remove the tuple from the original list
            if (100 - new_score) < threshold:
                curr_circuit.remove((layer, head))

                print("\nRemoved:", (layer, head))
                print(new_score)

    return curr_circuit, new_score

# Ablate the model and compare with original

### iter fwd backw, threshold 20

In [ ]:
threshold = 20
curr_circuit = []
prev_score = 100
new_score = 0
iter = 1
while prev_score != new_score:
    print('\nfwd prune, iter ', str(iter))
    # track changes in circuit as for some reason it doesn't work with scores
    old_circuit = curr_circuit.copy() # save old before finding new one
    curr_circuit, new_score = find_circuit_forw(curr_circuit=curr_circuit, threshold=threshold)
    if curr_circuit == old_circuit:
        break
    print('\nbackw prune, iter ', str(iter))
    # prev_score = new_score # save old score before finding new one
    old_circuit = curr_circuit.copy() # save old before finding new one
    curr_circuit, new_score = find_circuit_backw(curr_circuit=curr_circuit, threshold=threshold)
    if curr_circuit == old_circuit:
        break
    iter += 1


fwd prune, iter  1

Removed: (0, 0)
96.38060760498047

Removed: (0, 1)
82.5677719116211

Removed: (0, 2)
82.70421600341797

Removed: (0, 3)
81.77704620361328

Removed: (0, 4)
82.78357696533203

Removed: (0, 6)
85.57305145263672

Removed: (0, 7)
85.28573608398438

Removed: (0, 8)
81.6546630859375

Removed: (0, 11)
81.86619567871094

Removed: (1, 1)
80.1313705444336

Removed: (1, 2)
80.03694152832031

Removed: (1, 6)
80.6706314086914

Removed: (1, 7)
80.28878784179688

Removed: (1, 8)
82.18318939208984

Removed: (1, 9)
81.25663757324219

Removed: (1, 10)
83.76980590820312

Removed: (2, 3)
81.14581298828125

Removed: (2, 4)
81.83724212646484

Removed: (2, 11)
81.88544464111328

Removed: (3, 0)
83.1378173828125

Removed: (3, 1)
80.99207305908203

Removed: (3, 4)
81.99833679199219

Removed: (3, 8)
82.0475082397461

Removed: (3, 9)
80.53414916992188

Removed: (3, 10)
81.74751281738281

Removed: (4, 0)
81.10719299316406

Removed: (4, 3)
81.47115325927734

Removed: (4, 5)
81.57735443115234

R

In [ ]:
curr_circuit

[(0, 5),
 (1, 0),
 (1, 5),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 5),
 (2, 6),
 (2, 7),
 (2, 8),
 (2, 9),
 (3, 3),
 (3, 5),
 (3, 6),
 (3, 7),
 (3, 11),
 (4, 1),
 (4, 2),
 (4, 6),
 (4, 8),
 (4, 10),
 (5, 0),
 (5, 1),
 (5, 6),
 (5, 8),
 (5, 9),
 (6, 8),
 (6, 10),
 (7, 6),
 (7, 8),
 (7, 9),
 (7, 10),
 (7, 11),
 (8, 0),
 (8, 1),
 (8, 5),
 (8, 6),
 (8, 7),
 (8, 8),
 (9, 1),
 (9, 3),
 (9, 5),
 (9, 11),
 (10, 7),
 (11, 0),
 (11, 1),
 (11, 7),
 (11, 8),
 (11, 9)]

In [ ]:
fb_20 = curr_circuit.copy()
len(fb_20)

49

#### loop rmv and check for most impt heads

In [ ]:
circ = fb_20
mean_ablate_by_lst(circ, model, orig_score, print_output=True).item()

Average logit difference (circuit / full) %: 80.3439


80.34394836425781

In [ ]:
lh_scores = {}
for lh in circ:
    copy_circuit = circ.copy()
    copy_circuit.remove(lh)
    print("removed: " + str(lh))
    new_score = mean_ablate_by_lst(copy_circuit, model, orig_score, print_output=True).item()
    lh_scores[lh] = new_score

removed: (0, 5)
Average logit difference (circuit / full) %: 75.4070
removed: (1, 0)
Average logit difference (circuit / full) %: 60.9708
removed: (1, 5)
Average logit difference (circuit / full) %: 58.1626
removed: (2, 0)
Average logit difference (circuit / full) %: 78.0082
removed: (2, 1)
Average logit difference (circuit / full) %: 76.2738
removed: (2, 2)
Average logit difference (circuit / full) %: 44.5944
removed: (2, 5)
Average logit difference (circuit / full) %: 78.3988
removed: (2, 6)
Average logit difference (circuit / full) %: 73.2227
removed: (2, 7)
Average logit difference (circuit / full) %: 76.7953
removed: (2, 8)
Average logit difference (circuit / full) %: 73.8933
removed: (2, 9)
Average logit difference (circuit / full) %: 67.1837
removed: (3, 3)
Average logit difference (circuit / full) %: 51.4896
removed: (3, 5)
Average logit difference (circuit / full) %: 75.9370
removed: (3, 6)
Average logit difference (circuit / full) %: 68.8061
removed: (3, 7)
Average logit diff

In [ ]:
dict(sorted(lh_scores.items(), key=lambda item: item[1]))

{(4, 10): -7.344493865966797,
 (7, 11): 10.858027458190918,
 (7, 10): 43.18928527832031,
 (6, 10): 44.217735290527344,
 (2, 2): 44.59440994262695,
 (9, 5): 45.4736213684082,
 (5, 8): 46.33286666870117,
 (5, 1): 46.9207649230957,
 (3, 3): 51.489646911621094,
 (7, 6): 52.928993225097656,
 (3, 7): 53.48268508911133,
 (1, 5): 58.162567138671875,
 (8, 0): 58.78353500366211,
 (4, 6): 59.800315856933594,
 (6, 8): 60.119991302490234,
 (1, 0): 60.970829010009766,
 (5, 9): 61.743324279785156,
 (9, 3): 66.15119171142578,
 (2, 9): 67.18372344970703,
 (3, 6): 68.80607604980469,
 (8, 6): 69.43566131591797,
 (8, 5): 70.15142822265625,
 (11, 0): 70.94245910644531,
 (9, 1): 71.47088623046875,
 (11, 8): 72.57658386230469,
 (5, 0): 73.01366424560547,
 (2, 6): 73.22274780273438,
 (2, 8): 73.89334106445312,
 (5, 6): 74.00653839111328,
 (4, 8): 74.91043853759766,
 (8, 8): 74.96321868896484,
 (0, 5): 75.4069595336914,
 (3, 5): 75.93695068359375,
 (10, 7): 76.26075744628906,
 (2, 1): 76.27378845214844,
 (8, 7

In [ ]:
lh = (4, 10)
copy_circuit = circ.copy()
copy_circuit.remove(lh)

lst = copy_circuit
CIRCUIT = {
        "number mover": lst,
        # "number mover 10": lst,
        # "number mover 9": lst,
        "number mover 8": lst,
        "number mover 7": lst,
        "number mover 6": lst,
        "number mover 5": lst,
        "number mover 4": lst,
        "number mover 3": lst,
        "number mover 2": lst,
        "number mover 1": lst,
    }

SEQ_POS_TO_KEEP = {
    "number mover": "end",
    # "number mover 10": "S10",
    # "number mover 9": "S9",
    "number mover 8": "S8",
    "number mover 7": "S7",
    "number mover 6": "S6",
    "number mover 5": "S5",
    "number mover 4": "S4",
    "number mover 3": "S3",
    "number mover 2": "S2",
    "number mover 1": "S1",
}

model.reset_hooks(including_permanent=True)  #must do this after running with mean ablation hook
new_score = mean_ablate_by_lst(copy_circuit, model, orig_score, print_output=True).item()
model = ioi_circuit_extraction.add_mean_ablation_hook(model, means_dataset=dataset_2, circuit=CIRCUIT, seq_pos_to_keep=SEQ_POS_TO_KEEP)
ioi_logits_minimal = model(dataset.toks)
new_score = logits_to_ave_logit_diff_2(ioi_logits_minimal, dataset)
new_score

Average logit difference (circuit / full) %: -7.3445


tensor(-0.1915, device='cuda:0')

In [ ]:
orig_score

tensor(2.6070, device='cuda:0')

## iter backw fwd, threshold 20

In [ ]:
threshold = 20
curr_circuit = []
prev_score = 100
new_score = 0
iter = 1
while prev_score != new_score:
    print('\nbackw prune, iter ', str(iter))
    # prev_score = new_score # save old score before finding new one
    old_circuit = curr_circuit.copy() # save old before finding new one
    curr_circuit, new_score = find_circuit_backw(curr_circuit=curr_circuit, orig_score=orig_score, threshold=threshold)
    if curr_circuit == old_circuit:
        break
    print('\nfwd prune, iter ', str(iter))
    # track changes in circuit as for some reason it doesn't work with scores
    old_circuit = curr_circuit.copy() # save old before finding new one
    curr_circuit, new_score = find_circuit_forw(curr_circuit=curr_circuit, orig_score=orig_score, threshold=threshold)
    if curr_circuit == old_circuit:
        break
    iter += 1


backw prune, iter  1

Removed: (11, 0)
90.67725372314453

Removed: (11, 1)
89.80494689941406

Removed: (11, 2)
90.05421447753906

Removed: (11, 3)
89.86155700683594

Removed: (11, 4)
90.72091674804688

Removed: (11, 5)
90.60135650634766

Removed: (11, 6)
90.70639038085938

Removed: (11, 7)
89.79895782470703

Removed: (11, 8)
83.32293701171875

Removed: (11, 9)
81.96453857421875

Removed: (11, 10)
81.42296600341797

Removed: (11, 11)
81.58457946777344

Removed: (10, 0)
81.6201171875

Removed: (10, 1)
81.55277252197266

Removed: (10, 2)
87.96232604980469

Removed: (10, 3)
88.63429260253906

Removed: (10, 4)
87.46839141845703

Removed: (10, 5)
86.12212371826172

Removed: (10, 6)
86.17699432373047

Removed: (10, 8)
85.81008911132812

Removed: (10, 9)
85.0916519165039

Removed: (10, 10)
85.64619445800781

Removed: (10, 11)
84.75440979003906

Removed: (9, 0)
84.7319107055664

Removed: (9, 2)
84.59341430664062

Removed: (9, 4)
81.57080078125

Removed: (9, 6)
80.54354095458984

Removed: (9, 8

In [ ]:
bf_3 = curr_circuit.copy()
bf_3

[(1, 5),
 (1, 11),
 (2, 1),
 (2, 2),
 (2, 4),
 (2, 5),
 (2, 6),
 (2, 7),
 (2, 8),
 (2, 9),
 (3, 1),
 (3, 3),
 (3, 6),
 (3, 7),
 (3, 10),
 (3, 11),
 (4, 6),
 (4, 10),
 (4, 11),
 (5, 1),
 (5, 6),
 (5, 8),
 (5, 9),
 (6, 1),
 (6, 3),
 (6, 8),
 (6, 10),
 (7, 6),
 (7, 8),
 (7, 10),
 (7, 11),
 (8, 0),
 (8, 5),
 (8, 6),
 (8, 7),
 (8, 8),
 (9, 1),
 (9, 3),
 (9, 5),
 (9, 7),
 (10, 7)]

In [ ]:
len(bf_3)

41

### loop rmv and check for most impt heads

In [ ]:
circ = bf_3
circ_score = mean_ablate_by_lst(circ, model, orig_score, print_output=True).item()

Average logit difference (circuit / full) %: 80.1039


In [ ]:
lh_scores = {}
for lh in circ:
    copy_circuit = circ.copy()
    copy_circuit.remove(lh)
    print("removed: " + str(lh))
    new_score = mean_ablate_by_lst(copy_circuit, model, orig_score, print_output=True).item()
    lh_scores[lh] = new_score

removed: (1, 5)
Average logit difference (circuit / full) %: 57.7173
removed: (1, 11)
Average logit difference (circuit / full) %: 75.8694
removed: (2, 1)
Average logit difference (circuit / full) %: 75.5749
removed: (2, 2)
Average logit difference (circuit / full) %: 64.7992
removed: (2, 4)
Average logit difference (circuit / full) %: 79.0984
removed: (2, 5)
Average logit difference (circuit / full) %: 79.1150
removed: (2, 6)
Average logit difference (circuit / full) %: 75.8213
removed: (2, 7)
Average logit difference (circuit / full) %: 77.7041
removed: (2, 8)
Average logit difference (circuit / full) %: 78.5678
removed: (2, 9)
Average logit difference (circuit / full) %: 72.8585
removed: (3, 1)
Average logit difference (circuit / full) %: 76.7338
removed: (3, 3)
Average logit difference (circuit / full) %: 64.5689
removed: (3, 6)
Average logit difference (circuit / full) %: 71.0821
removed: (3, 7)
Average logit difference (circuit / full) %: 64.9705
removed: (3, 10)
Average logit di

In [ ]:
sorted_lh_scores = dict(sorted(lh_scores.items(), key=lambda item: item[1]))
sorted_lh_scores

{(4, 10): -7.2874627113342285,
 (7, 11): 15.494046211242676,
 (7, 10): 41.55342102050781,
 (5, 1): 41.56138229370117,
 (5, 8): 48.6051139831543,
 (4, 11): 48.80915451049805,
 (6, 10): 53.62663269042969,
 (9, 5): 54.65618896484375,
 (7, 6): 57.5998420715332,
 (1, 5): 57.71733093261719,
 (8, 0): 63.006248474121094,
 (3, 10): 63.10055923461914,
 (3, 3): 64.56888580322266,
 (2, 2): 64.79924774169922,
 (3, 7): 64.97046661376953,
 (5, 9): 65.6832275390625,
 (6, 8): 65.90373992919922,
 (4, 6): 66.96007537841797,
 (9, 3): 70.00293731689453,
 (3, 6): 71.08211517333984,
 (2, 9): 72.85848236083984,
 (8, 5): 73.31848907470703,
 (8, 6): 73.32933807373047,
 (9, 1): 74.13877868652344,
 (2, 1): 75.57490539550781,
 (2, 6): 75.8212661743164,
 (1, 11): 75.86937713623047,
 (8, 8): 76.13774871826172,
 (6, 3): 76.39346313476562,
 (3, 1): 76.73382568359375,
 (9, 7): 77.01638793945312,
 (8, 7): 77.2037353515625,
 (10, 7): 77.48711395263672,
 (2, 7): 77.70409393310547,
 (3, 11): 77.94585418701172,
 (6, 1): 78.

In [ ]:
for lh, score in sorted_lh_scores.items():
    print(lh, -round(circ_score-score, 2))

(4, 10) -87.39
(7, 11) -64.61
(7, 10) -38.55
(5, 1) -38.54
(5, 8) -31.5
(4, 11) -31.29
(6, 10) -26.48
(9, 5) -25.45
(7, 6) -22.5
(1, 5) -22.39
(8, 0) -17.1
(3, 10) -17.0
(3, 3) -15.54
(2, 2) -15.3
(3, 7) -15.13
(5, 9) -14.42
(6, 8) -14.2
(4, 6) -13.14
(9, 3) -10.1
(3, 6) -9.02
(2, 9) -7.25
(8, 5) -6.79
(8, 6) -6.77
(9, 1) -5.97
(2, 1) -4.53
(2, 6) -4.28
(1, 11) -4.23
(8, 8) -3.97
(6, 3) -3.71
(3, 1) -3.37
(9, 7) -3.09
(8, 7) -2.9
(10, 7) -2.62
(2, 7) -2.4
(3, 11) -2.16
(6, 1) -2.1
(2, 8) -1.54
(5, 6) -1.06
(2, 4) -1.01
(7, 8) -0.99
(2, 5) -0.99


In [ ]:
# circ = [(1, 5), (1, 11), (2, 1), (2, 2), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (3, 1), (3, 3), (3, 6), (3, 7), (3, 10), (3, 11), (4, 6), (4, 10), (4, 11), (5, 1), (5, 6), (5, 8), (5, 9), (6, 1), (6, 3), (6, 8), (6, 10), (7, 6), (7, 8), (7, 10), (7, 11), (8, 0), (8, 5), (8, 6), (8, 7), (8, 8), (9, 1), (9, 3), (9, 5), (9, 7), (10, 7)]
# remove (4, 10)
circ = [(1, 5), (1, 11), (2, 1), (2, 2), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (3, 1), (3, 3), (3, 6), (3, 7), (3, 10), (3, 11), (4, 6), (4, 11), (5, 1), (5, 6), (5, 8), (5, 9), (6, 1), (6, 3), (6, 8), (6, 10), (7, 6), (7, 8), (7, 10), (7, 11), (8, 0), (8, 5), (8, 6), (8, 7), (8, 8), (9, 1), (9, 3), (9, 5), (9, 7), (10, 7)]
circ_score = mean_ablate_by_lst(circ, model, orig_score, print_output=True).item()

Average logit difference (circuit / full) %: -7.2875


#### after ipp

In [25]:
circ = [(1, 5), (1, 11), (2, 1), (2, 2), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (3, 1), (3, 3), (3, 6), (3, 7), (3, 10), (3, 11), (4, 6), (4, 10), (4, 11), (5, 1), (5, 6), (5, 8), (5, 9), (6, 1), (6, 3), (6, 8), (6, 10), (7, 6), (7, 8), (7, 10), (7, 11), (8, 0), (8, 5), (8, 6), (8, 7), (8, 8), (9, 1), (9, 3), (9, 5), (9, 7), (10, 7)]
decr_adjList_copy = circ.copy()
tuples_to_remove = [(3, 1), (2, 8), (9, 7), (2, 4), (9, 3), (7, 8)]
decr_adjList_copy = [t for t in decr_adjList_copy if t not in tuples_to_remove]

circ_score = mean_ablate_by_lst(decr_adjList_copy, model, orig_score, print_output=True).item()

Average logit difference (circuit / full) %: 60.7876


### try other tasks circs

### bf 80

In [ ]:
# # fb 80, digits incr
# # https://colab.research.google.com/drive/1mFWmGAKtigFcqqWWMCwU7wWQY2HT5ZOo#scrollTo=lJEY-Zs2g_a5&line=1&uniqifier=1
# circuit = [(1, 5), (3, 3), (3, 7), (3, 10), (3, 11), (4, 4), (4, 6), (4, 7), (4, 10), (4, 11), (5, 0), (5, 2), (5, 3), (5, 4), (5, 6), (6, 3), (6, 8), (6, 10), (7, 0), (7, 2), (7, 7), (7, 8), (7, 10), (7, 11), (8, 0), (8, 1), (8, 6), (8, 8), (8, 9), (8, 11), (9, 1), (11, 8)]
# mean_ablate_by_lst(circuit, model, print_output=True).item()

In [ ]:
# # fb 80, numwords
# # https://colab.research.google.com/drive/1QTv-4osLHadCAay0beew-xlXszPCG88s#scrollTo=563kZf_4r_mw&line=2&uniqifier=1
# circuit = [(3, 2), (4, 4), (4, 8), (4, 10), (4, 11), (5, 5), (5, 6), (5, 7), (5, 8), (6, 1), (6, 7), (6, 9), (6, 10), (7, 0), (7, 2), (7, 5), (7, 6), (7, 7), (7, 8), (7, 10), (7, 11), (8, 0), (8, 1), (8, 6), (8, 8), (8, 11), (9, 1), (10, 2)]
# mean_ablate_by_lst(circuit, model, print_output=True).item()

In [ ]:
# # fb 80, months
# # https://colab.research.google.com/drive/1lhQqlizYGMC11vzp6I9mJ3dyxIr8tV3l#scrollTo=563kZf_4r_mw&line=2&uniqifier=1
# circuit = [(4, 4), (7, 11), (8, 6), (8, 9), (8, 11), (9, 1), (9, 5), (11, 10)]
# mean_ablate_by_lst(circuit, model, print_output=True).item()

## bf 97

In [ ]:
# digits incr
# incorr i+3
circuit = [(0, 1), (0, 2), (0, 5), (0, 7), (0, 8), (0, 10), (1, 0), (1, 1), (1, 3), (1, 5), (1, 7), (1, 11), (2, 0), (2, 1), (2, 2), (2, 3), (2, 5), (2, 6), (2, 8), (2, 9), (2, 10), (3, 3), (3, 7), (3, 8), (3, 10), (3, 11), (4, 2), (4, 4), (4, 6), (4, 10), (4, 11), (5, 1), (5, 4), (5, 8), (5, 10), (5, 11), (6, 2), (6, 3), (6, 4), (6, 6), (6, 7), (6, 8), (6, 9), (6, 10), (6, 11), (7, 11), (8, 6), (8, 8), (9, 1), (10, 7), (11, 10)]
mean_ablate_by_lst(circuit, model, orig_score, print_output=True).item()

Average logit difference (circuit / full) %: 6.2192


6.219162464141846

In [ ]:
# numwords
# incorr i+3
circuit = [(0, 1), (0, 6), (0, 7), (0, 9), (0, 10), (1, 0), (1, 5), (3, 3), (4, 4), (4, 10), (5, 4), (5, 6), (5, 8), (6, 6), (6, 10), (7, 6), (7, 10), (7, 11), (8, 8), (9, 1), (10, 7)]
mean_ablate_by_lst(circuit, model, orig_score, print_output=True).item()

Average logit difference (circuit / full) %: 1.5315


1.53154718875885

In [ ]:
# months
# incorr i
circuit = [(0, 1), (2, 3), (2, 5), (2, 7), (2, 8), (2, 9), (4, 4), (5, 0), (5, 6), (6, 9), (6, 10), (7, 8), (7, 11), (8, 1), (8, 6), (8, 8), (8, 9), (9, 1), (9, 7), (9, 11), (10, 7), (11, 10)]
mean_ablate_by_lst(circuit, model, orig_score, print_output=True).item()

Average logit difference (circuit / full) %: 0.8478


0.8477634787559509